# Demo: STAC API POST /search endpoint example

This notebook is meant to be a very short and simple demo on how to do a simple search in swisstopo's STAC API and do a quick visualization of the downloaded data.<br>
Here are some useful links, you might be interested in:<br>
- [STAC SPEC ("Speck-Bsteck" ;-))](https://stacspec.org/en)
- [STAC API SPEC](https://github.com/radiantearth/stac-api-spec)
- [swisstopo's STAC API](https://data.geo.admin.ch/api/stac/v0.9/)
- [swisstopo's STAC API docu/spec](https://data.geo.admin.ch/api/stac/static/spec/v0.9/api.html)

**Note: To keep this notebook short and simple, some steps/best practices, such as proper error handling or checking checksums of downloaded files, are not implemented to a full extent**

## Preparations
First we need to import the necessary modules. <br>
Make sure you have installed all the dependencies, e.g. by running: `pip install folium geopandas jupyter mapclassify matplotlib pandas requests xyzservices`,<br>
or e.g. by creating a [virtual environment](https://docs.python-guide.org/dev/virtualenvs/) using the `Pipfile`, which is contained in the same github directory, as this notebook.

In [ ]:
import json
import os
import pprint

import geopandas as gpd
import requests

We will use the [API's POST /search endpoint](https://data.geo.admin.ch/api/stac/static/spec/v0.9/api.html#tag/STAC/operation/postSearchSTAC) for our search.<br>
For this we need to define, where (i.e. in which collection(s)) and what (i.e. search term) we want to search.<br>
Lets try:
- search for items, which contain "2023" in their title
- search in the collection "ch.are.agglomerationsverkehr" (also check the according [metadata](https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/f4b72bb8-aff0-4eab-b1e8-48e698c0e8fb))<br>
- lets limit the returned results to only 1 ( = "only the first hit"), to make things less complicated ;-)<br>
<br>
Note: You could also do more advanced searches, using bounding boxes or limiting to time intervals, and the like.

In [ ]:
search_string = "2023"
search_collection = "ch.are.agglomerationsverkehr"
limit = 1

payload = {
    "limit": limit,
    "query": {
        "title": {
            "contains": search_string
        },
    },
    "collections": [search_collection]
}


try:
    response = requests.post("https://data.geo.admin.ch/api/stac/v0.9/search",
                             data=json.dumps(payload), headers={"Content-Type": "application/json"})
# I know, this is a "too broad exception" clause ;-)
# For a demo, this will do. But of course, one should be more specific here and
# catch different exceptions specifically
except Exception as e:
    raise(e)

Now we can check, what we got back from the STAC API.
If everything went well, we should have something very similar to the "Response samples" from the POST /search endpoint's documentation.

In [ ]:
response.json()

From the returned "FeatureCollection" we can now extract the details of the first (and only, since we used "limit=1") item can be extracted.<br>
**(Again, for the sake of simplicity, there's no error handling implemented here. But in real life it might well make sense, to check the response of our request. Is there any data returned? Which format, etc. Otherwise there might be exceptions when trying to access certain elements of the response, which don't exist)**

In [ ]:
# if we had more than one item, we could use a loop over all items here.
# In this demo, we only have one item
item = response.json()["features"][0]
assets = item["assets"]


Now, `assets` contains a list of all assets belonging to the item that our search returned.<br>
Let's check, how `assets` looks like

In [ ]:
pprint.pprint(assets)

There is only one assest, a .gpkg, belonging to our item. We'll use the `list(dict.items())` trick here, to access the data we need.<br>
Again, if we had more than only one asset, we could do everything in a loop over all the assets.

In [ ]:
asset_name = list(assets.items())[0][0]
asset_data = list(assets.items())[0][1]
asset_href = asset_data["href"]

Now we have the asset's href and can download it from the STAC API.<br>
Note: In the code below, we will only download the asset, in case it does not exist locally already.<br>
For our demo and especially for very large files this makes sense, as it won't download the file twice.<br>
**In real life you might want to check, if the local version is old, and the STAC API could offer a newer version, which you might want to download.**
<br>
<br>
Additionally, in our case we are dealing with a .pgkp file. In other cases, the API might return a zipped file, which would need to be extracted after downloading. This would require adaptions of the code below.<br>
And, last but not least, in real life you'd want to check if the checksum of the downloaded file matches the one returned by the API.

In [ ]:
fname = os.path.basename(asset_href)
# define the local path of the asset file
path = f"./data/{asset_name.split('.', 1)[0]}/{asset_name}"

# create the necessary directories, if not existing
if not os.path.exists(f"./data/{asset_name.split('.', 1)[0]}"):
    os.makedirs(f"./data/{asset_name.split('.', 1)[0]}")

# only send the request, if asset file does not yet exist locally
# useful for large files ;-)
# Again: in real life, you'd want to do proper exception handling here, too, as well
# as check if the checksum of the download matches the one specified in the API's response.
if not os.path.exists(path):
    req = requests.get(asset_href)
    with open(path, 'wb') as outfile:
        outfile.write(req.content)


Now we can simply load our asset file into a geopandas GeoDataFrame

In [ ]:
data = gpd.read_file(path)
data

And finally, with only one line of code, have a quick look at the asset to see, if it is what we expected, while using a swisstopo map as background ;-)<br>
This is done using geopanda's [geopandas.GeoDataFrame.explore](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html#geopandas-geodataframe-explore), which creates an interactive leaflet map.

In [ ]:
data.explore("Name", legend=False, tiles="SwissFederalGeoportal NationalMapColor", attr="© Data:swisstopo")